In [6]:
import os
import random
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Function to preprocess the images
def preprocess_image(img_path, target_size=(120, 120)):
    """
    Preprocess the image by resizing it and normalizing it between 0 and 1.
    
    Parameters:
    - img_path: str, path to the image file.
    - target_size: tuple, target size for resizing.
    
    Returns:
    - Preprocessed image as a numpy array.
    """
    img = load_img(img_path, target_size=target_size)
    img = img_to_array(img) / 255.0  # Normalize the image to range [0, 1]
    return img

# Function to generate support and query sets for people with at least 2 images
def create_support_query_sets(dataset_path, person_folders, images_by_person, num_support=1):
    """
    Create support and query sets for each person based on available images.

    Parameters:
    - dataset_path: str, path to the dataset containing the person folders.
    - person_folders: list of str, list of person names or folder names.
    - images_by_person: dict, mapping of person to list of their image paths.
    - num_support: int, number of support images to pick for each person (default 1).

    Returns:
    - support_set: dict, mapping of person to their support image(s).
    - query_set: dict, mapping of person to their query image(s).
    """
    support_set = {}
    query_set = {}

    # Loop over each person
    for person in person_folders:
        images = images_by_person.get(person, [])
        
        # Consider people with at least 2 images
        if len(images)==2:
            support_images = random.sample(images, num_support)  # Pick images for support
            query_images = [img for img in images if img not in support_images]  # Remaining images for query

            support_set[person] = support_images
            query_set[person] = query_images

    return support_set, query_set

# Example usage:
# Assuming `dataset_path` is the directory containing subdirectories for each person,
# and `images_by_person` is a dictionary that maps each person to their list of image paths.

# Example: Dataset structure
dataset_path = '/Users/shuchanzhou/Downloads/archive-2/lfw-deepfunneled/lfw-deepfunneled'

person_folders = os.listdir(dataset_path)  # Get all person folders in the dataset

# Create a dictionary mapping each person to their list of image paths
images_by_person = {}
for person in person_folders:
    person_folder_path = os.path.join(dataset_path, person)
    if os.path.isdir(person_folder_path):
        images = [os.path.join(person_folder_path, img) for img in os.listdir(person_folder_path) if img.endswith(('jpg', 'jpeg', 'png'))]
        images_by_person[person] = images

# Now, create the support and query sets
support_set, query_set = create_support_query_sets(dataset_path, person_folders, images_by_person)

In [7]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics.pairwise import cosine_similarity

def compare_embeddings1(query_embedding, support_embedding):
    """
    Compare the query embedding with support embeddings and return the cosine distance score.

    Parameters:
    - query_embedding: The embedding of the query image
    - support_embedding: The embedding of a support image
    
    Returns:
    - Cosine distance between the two embeddings (lower distance means more similar)
    """
    cosine_sim = cosine_similarity([query_embedding], [support_embedding])[0][0]
    cosine_dist = 1 - cosine_sim  # Convert similarity to distance
    return cosine_dist



def compare_embeddings(query_embedding, support_embedding):
    """
    Compare the query embedding with support embeddings and return the similarity score.

    Parameters:
    - query_embedding: The embedding of the query image
    - support_embedding: The embedding of a support image
    
    Returns:
    - Distance between the two embeddings (lower distance means more similar)
    """
    return euclidean_distances([query_embedding], [support_embedding])[0][0]
    

def few_shot_inference(support_embeddings, query_embeddings):
    """
    Perform few-shot learning inference by comparing query images with the support set.

    Parameters:
    - support_embeddings: Dictionary of support embeddings for each person
    - query_embeddings: Dictionary of query embeddings for each person

    Returns:
    - correct_predictions: Number of correct predictions
    - total_predictions: Total number of predictions made
    - accuracy: Accuracy of the model on the query set
    """
    correct_predictions = 0
    total_predictions = 0

    for person in query_embeddings:
        for query_embedding in query_embeddings[person]:
            min_distance = float('inf')
            predicted_person = None

            # Compare query image with all support images (from all people)
            for support_person, support_images in support_embeddings.items():
                for support_embedding in support_images:
                    # Compare query embedding with support embedding
                    distance = compare_embeddings1(query_embedding, support_embedding)
                

                    # If smaller distance, update prediction
                    if distance < min_distance:
                        min_distance = distance
                        predicted_person = support_person

            # Check if the prediction matches the actual person
            if predicted_person == person:
                correct_predictions += 1
            total_predictions += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy, correct_predictions, total_predictions

In [8]:
from deepface import DeepFace
import pandas as pd

def generate_embeddings_for_few_shot(model, support_set, query_set):
    """
    Generate embeddings for both the support and query sets.

    Parameters:
    - model: The model used for face verification (e.g., 'Facenet', 'VGG-Face', etc.)
    - support_set: Dictionary mapping person to their support images
    - query_set: Dictionary mapping person to their query images

    Returns:
    - support_embeddings: Dictionary of embeddings for the support images
    - query_embeddings: Dictionary of embeddings for the query images
    """
    support_embeddings = {}
    query_embeddings = {}
    
    # Generate embeddings for the support set
    for person, images in support_set.items():
        embeddings = []
        for img in images:
            result = DeepFace.represent(img, model_name=model,enforce_detection=False)
            embedding = result[0]['embedding']  # Extract embedding from the result
            embeddings.append(embedding)
        support_embeddings[person] = embeddings
        

    # Generate embeddings for the query set
    for person, images in query_set.items():
        embeddings = []
        for img in images:
            result = DeepFace.represent(img, model_name=model,enforce_detection=False)
            embedding = result[0]['embedding']  # Extract embedding from the result
            embeddings.append(embedding)
        query_embeddings[person] = embeddings

    return support_embeddings, query_embeddings

## one shot for n=2; 779 pictures in total

In [9]:
model = 'Facenet'  # or any other model name that DeepFace supports

# Generate embeddings for both support and query sets
support_embeddings, query_embeddings = generate_embeddings_for_few_shot(model, support_set, query_set)

# Perform few-shot learning and get accuracy
accuracy, correct_predictions, total_predictions = few_shot_inference(support_embeddings, query_embeddings)

# Output results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

Accuracy: 66.50%
Correct Predictions: 518/779


In [10]:
model = 'Facenet512'   # or any other model name that DeepFace supports

# Generate embeddings for both support and query sets
support_embeddings, query_embeddings = generate_embeddings_for_few_shot(model, support_set, query_set)

# Perform few-shot learning and get accuracy
accuracy, correct_predictions, total_predictions = few_shot_inference(support_embeddings, query_embeddings)

# Output results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

Accuracy: 72.91%
Correct Predictions: 568/779


In [11]:
model = 'GhostFaceNet'   # or any other model name that DeepFace supports

# Generate embeddings for both support and query sets
support_embeddings, query_embeddings = generate_embeddings_for_few_shot(model, support_set, query_set)

# Perform few-shot learning and get accuracy
accuracy, correct_predictions, total_predictions = few_shot_inference(support_embeddings, query_embeddings)

# Output results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

24-11-30 17:05:45 - Pre-trained weights is downloaded from https://github.com/HamadYA/GhostFaceNets/releases/download/v1.2/GhostFaceNet_W1.3_S1_ArcFace.h5 to /Users/shuchanzhou/.deepface/weights/ghostfacenet_v1.h5


Downloading...
From: https://github.com/HamadYA/GhostFaceNets/releases/download/v1.2/GhostFaceNet_W1.3_S1_ArcFace.h5
To: /Users/shuchanzhou/.deepface/weights/ghostfacenet_v1.h5
100%|█████████████████████████████████████████████████████████████████| 17.3M/17.3M [00:00<00:00, 17.3MB/s]


24-11-30 17:05:47 - Pre-trained weights is just downloaded to /Users/shuchanzhou/.deepface/weights/ghostfacenet_v1.h5
Accuracy: 65.34%
Correct Predictions: 509/779


In [12]:
model = 'OpenFace'   # or any other model name that DeepFace supports

# Generate embeddings for both support and query sets
support_embeddings, query_embeddings = generate_embeddings_for_few_shot(model, support_set, query_set)

# Perform few-shot learning and get accuracy
accuracy, correct_predictions, total_predictions = few_shot_inference(support_embeddings, query_embeddings)

# Output results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

24-11-30 17:12:29 - openface_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/openface_weights.h5
To: /Users/shuchanzhou/.deepface/weights/openface_weights.h5
100%|█████████████████████████████████████████████████████████████████| 15.3M/15.3M [00:00<00:00, 19.4MB/s]


Accuracy: 6.42%
Correct Predictions: 50/779


In [13]:
model = 'DeepID'   # or any other model name that DeepFace supports

# Generate embeddings for both support and query sets
support_embeddings, query_embeddings = generate_embeddings_for_few_shot(model, support_set, query_set)

# Perform few-shot learning and get accuracy
accuracy, correct_predictions, total_predictions = few_shot_inference(support_embeddings, query_embeddings)

# Output results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

24-11-30 17:16:20 - deepid_keras_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/deepid_keras_weights.h5
To: /Users/shuchanzhou/.deepface/weights/deepid_keras_weights.h5
100%|█████████████████████████████████████████████████████████████████| 1.61M/1.61M [00:00<00:00, 4.88MB/s]


Accuracy: 6.03%
Correct Predictions: 47/779


In [14]:
model = 'SFace'   # or any other model name that DeepFace supports

# Generate embeddings for both support and query sets
support_embeddings, query_embeddings = generate_embeddings_for_few_shot(model, support_set, query_set)

# Perform few-shot learning and get accuracy
accuracy, correct_predictions, total_predictions = few_shot_inference(support_embeddings, query_embeddings)

# Output results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

24-11-30 17:19:51 - face_recognition_sface_2021dec.onnx weights will be downloaded...


Downloading...
From: https://github.com/opencv/opencv_zoo/raw/main/models/face_recognition_sface/face_recognition_sface_2021dec.onnx
To: /Users/shuchanzhou/.deepface/weights/face_recognition_sface_2021dec.onnx
100%|█████████████████████████████████████████████████████████████████| 38.7M/38.7M [00:01<00:00, 27.4MB/s]


Accuracy: 63.03%
Correct Predictions: 491/779


## one shot for n=2,3; 1261 pictures in total

In [1]:
import os
import random
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Function to preprocess the images
def preprocess_image(img_path, target_size=(120, 120)):
    """
    Preprocess the image by resizing it and normalizing it between 0 and 1.
    
    Parameters:
    - img_path: str, path to the image file.
    - target_size: tuple, target size for resizing.
    
    Returns:
    - Preprocessed image as a numpy array.
    """
    img = load_img(img_path, target_size=target_size)
    img = img_to_array(img) / 255.0  # Normalize the image to range [0, 1]
    return img

# Function to generate support and query sets for people with at least 2 images
def create_support_query_sets(dataset_path, person_folders, images_by_person, num_support=1):
    """
    Create support and query sets for each person based on available images.

    Parameters:
    - dataset_path: str, path to the dataset containing the person folders.
    - person_folders: list of str, list of person names or folder names.
    - images_by_person: dict, mapping of person to list of their image paths.
    - num_support: int, number of support images to pick for each person (default 1).

    Returns:
    - support_set: dict, mapping of person to their support image(s).
    - query_set: dict, mapping of person to their query image(s).
    """
    support_set = {}
    query_set = {}

    # Loop over each person
    for person in person_folders:
        images = images_by_person.get(person, [])
        
        # Consider people with at least 2 images
        if len(images)==2 or len(images)==3:
            support_images = random.sample(images, num_support)  # Pick images for support
            query_images = [img for img in images if img not in support_images]  # Remaining images for query

            support_set[person] = support_images
            query_set[person] = query_images

    return support_set, query_set

# Example usage:
# Assuming `dataset_path` is the directory containing subdirectories for each person,
# and `images_by_person` is a dictionary that maps each person to their list of image paths.

# Example: Dataset structure
dataset_path = '/Users/shuchanzhou/Downloads/archive-2/lfw-deepfunneled/lfw-deepfunneled'

person_folders = os.listdir(dataset_path)  # Get all person folders in the dataset

# Create a dictionary mapping each person to their list of image paths
images_by_person = {}
for person in person_folders:
    person_folder_path = os.path.join(dataset_path, person)
    if os.path.isdir(person_folder_path):
        images = [os.path.join(person_folder_path, img) for img in os.listdir(person_folder_path) if img.endswith(('jpg', 'jpeg', 'png'))]
        images_by_person[person] = images

# Now, create the support and query sets
support_set, query_set = create_support_query_sets(dataset_path, person_folders, images_by_person)

In [2]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics.pairwise import cosine_similarity

def compare_embeddings1(query_embedding, support_embedding):
    """
    Compare the query embedding with support embeddings and return the cosine distance score.

    Parameters:
    - query_embedding: The embedding of the query image
    - support_embedding: The embedding of a support image
    
    Returns:
    - Cosine distance between the two embeddings (lower distance means more similar)
    """
    cosine_sim = cosine_similarity([query_embedding], [support_embedding])[0][0]
    cosine_dist = 1 - cosine_sim  # Convert similarity to distance
    return cosine_dist



def compare_embeddings(query_embedding, support_embedding):
    """
    Compare the query embedding with support embeddings and return the similarity score.

    Parameters:
    - query_embedding: The embedding of the query image
    - support_embedding: The embedding of a support image
    
    Returns:
    - Distance between the two embeddings (lower distance means more similar)
    """
    return euclidean_distances([query_embedding], [support_embedding])[0][0]
    

def few_shot_inference(support_embeddings, query_embeddings):
    """
    Perform few-shot learning inference by comparing query images with the support set.

    Parameters:
    - support_embeddings: Dictionary of support embeddings for each person
    - query_embeddings: Dictionary of query embeddings for each person

    Returns:
    - correct_predictions: Number of correct predictions
    - total_predictions: Total number of predictions made
    - accuracy: Accuracy of the model on the query set
    """
    correct_predictions = 0
    total_predictions = 0

    for person in query_embeddings:
        for query_embedding in query_embeddings[person]:
            min_distance = float('inf')
            predicted_person = None

            # Compare query image with all support images (from all people)
            for support_person, support_images in support_embeddings.items():
                for support_embedding in support_images:
                    # Compare query embedding with support embedding
                    distance = compare_embeddings1(query_embedding, support_embedding)
                

                    # If smaller distance, update prediction
                    if distance < min_distance:
                        min_distance = distance
                        predicted_person = support_person

            # Check if the prediction matches the actual person
            if predicted_person == person:
                correct_predictions += 1
            total_predictions += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy, correct_predictions, total_predictions

In [3]:
from deepface import DeepFace
import pandas as pd

def generate_embeddings_for_few_shot(model, support_set, query_set):
    """
    Generate embeddings for both the support and query sets.

    Parameters:
    - model: The model used for face verification (e.g., 'Facenet', 'VGG-Face', etc.)
    - support_set: Dictionary mapping person to their support images
    - query_set: Dictionary mapping person to their query images

    Returns:
    - support_embeddings: Dictionary of embeddings for the support images
    - query_embeddings: Dictionary of embeddings for the query images
    """
    support_embeddings = {}
    query_embeddings = {}
    
    # Generate embeddings for the support set
    for person, images in support_set.items():
        embeddings = []
        for img in images:
            result = DeepFace.represent(img, model_name=model,enforce_detection=False)
            embedding = result[0]['embedding']  # Extract embedding from the result
            embeddings.append(embedding)
        support_embeddings[person] = embeddings
        

    # Generate embeddings for the query set
    for person, images in query_set.items():
        embeddings = []
        for img in images:
            result = DeepFace.represent(img, model_name=model,enforce_detection=False)
            embedding = result[0]['embedding']  # Extract embedding from the result
            embeddings.append(embedding)
        query_embeddings[person] = embeddings

    return support_embeddings, query_embeddings

In [4]:
model = 'Facenet512'   # or any other model name that DeepFace supports

# Generate embeddings for both support and query sets
support_embeddings, query_embeddings = generate_embeddings_for_few_shot(model, support_set, query_set)

# Perform few-shot learning and get accuracy
accuracy, correct_predictions, total_predictions = few_shot_inference(support_embeddings, query_embeddings)

# Output results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

Accuracy: 69.51%
Correct Predictions: 946/1361


## one shot for n=2,3,4; 1922 pictures in total

In [7]:
import os
import random
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Function to preprocess the images
def preprocess_image(img_path, target_size=(120, 120)):
    """
    Preprocess the image by resizing it and normalizing it between 0 and 1.
    
    Parameters:
    - img_path: str, path to the image file.
    - target_size: tuple, target size for resizing.
    
    Returns:
    - Preprocessed image as a numpy array.
    """
    img = load_img(img_path, target_size=target_size)
    img = img_to_array(img) / 255.0  # Normalize the image to range [0, 1]
    return img

# Function to generate support and query sets for people with at least 2 images
def create_support_query_sets(dataset_path, person_folders, images_by_person, num_support=1):
    """
    Create support and query sets for each person based on available images.

    Parameters:
    - dataset_path: str, path to the dataset containing the person folders.
    - person_folders: list of str, list of person names or folder names.
    - images_by_person: dict, mapping of person to list of their image paths.
    - num_support: int, number of support images to pick for each person (default 1).

    Returns:
    - support_set: dict, mapping of person to their support image(s).
    - query_set: dict, mapping of person to their query image(s).
    """
    support_set = {}
    query_set = {}

    # Loop over each person
    for person in person_folders:
        images = images_by_person.get(person, [])
        
        # Consider people with at least 2 images
        if len(images)==2 or len(images)==3 or len(images)==4:
            support_images = random.sample(images, num_support)  # Pick images for support
            query_images = [img for img in images if img not in support_images]  # Remaining images for query

            support_set[person] = support_images
            query_set[person] = query_images

    return support_set, query_set

# Example usage:
# Assuming `dataset_path` is the directory containing subdirectories for each person,
# and `images_by_person` is a dictionary that maps each person to their list of image paths.

# Example: Dataset structure
dataset_path = '/Users/shuchanzhou/Downloads/archive-2/lfw-deepfunneled/lfw-deepfunneled'

person_folders = os.listdir(dataset_path)  # Get all person folders in the dataset

# Create a dictionary mapping each person to their list of image paths
images_by_person = {}
for person in person_folders:
    person_folder_path = os.path.join(dataset_path, person)
    if os.path.isdir(person_folder_path):
        images = [os.path.join(person_folder_path, img) for img in os.listdir(person_folder_path) if img.endswith(('jpg', 'jpeg', 'png'))]
        images_by_person[person] = images

# Now, create the support and query sets
support_set, query_set = create_support_query_sets(dataset_path, person_folders, images_by_person)

In [8]:
model = 'Facenet512'   # or any other model name that DeepFace supports

# Generate embeddings for both support and query sets
support_embeddings, query_embeddings = generate_embeddings_for_few_shot(model, support_set, query_set)

# Perform few-shot learning and get accuracy
accuracy, correct_predictions, total_predictions = few_shot_inference(support_embeddings, query_embeddings)

# Output results
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

Accuracy: 67.85%
Correct Predictions: 1304/1922
